In [ ]:
import h5py
import numpy as np
from rdkit import Chem
from rdkit.Chem import AllChem, DataStructs
import seaborn as sns
import pandas as pd
import pickle

## True FP vs training set

In [ ]:
mols_file = "../../data/paired_spectra/canopus_train_public/labels.tsv"
split_file = "../../data/paired_spectra/canopus_train_public/splits/canopus_hplus_100_0.csv"

all_mols = pd.read_csv(mols_file, sep="\t")
split = pd.read_csv(split_file)

train = split[split["Fold_100_0"] == "train"]
train_mols = all_mols[all_mols["spec"].isin(train["name"])]

assert len(train) == len(train_mols)

In [ ]:
def smiles_to_fp(smiles):
    mol = Chem.MolFromSmiles(smiles)
    fp = AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=4096)
    return fp

In [ ]:
train_mols["fp"] = train_mols["smiles"].apply(smiles_to_fp)

In [ ]:
# casmi_file = "../../data/paired_spectra/casmi2022/labels_true.tsv"
# casmi_mols = pd.read_csv(casmi_file, sep="\t")
casmi_file = "../../data/paired_spectra/casmi2022/casmi_classyfire_results.pkl"
casmi_mols = pd.read_pickle(casmi_file)
casmi_mols["fp"] = casmi_mols["smiles"].apply(smiles_to_fp)

In [ ]:
# Find the most similar molecule in the training set for each CASMI molecule by Tanimoto similarity
casmi_mols["train_true_most_similar"] = None
casmi_mols["train_true_similarity"] = None
for i, row in casmi_mols.iterrows():
    fp = row["fp"]
    similarities = train_mols["fp"].apply(
        lambda x: DataStructs.TanimotoSimilarity(fp, x)
    )
    max_sim = similarities.max()
    max_idx = similarities.idxmax()
    casmi_mols.at[i, "train_true_most_similar"] = train_mols.loc[
        max_idx, "smiles"
    ]
    casmi_mols.at[i, "train_true_similarity"] = max_sim

In [ ]:
# Plot the distribution of similarities
import matplotlib.pyplot as plt

plt.hist(casmi_mols["train_true_similarity"], bins=20)
plt.xlabel("Tanimoto similarity")
plt.ylabel("Count")
plt.title("Similarity between true FP and training set")
plt.show()

## predicted FP vs training set

In [ ]:
pred_file = "../../results/retrained_models/contrast_model_aug/retrieval_casmi2022/retrieval_canopus_hplus_100_0.p"
hdf_file = "../../data/paired_spectra/casmi2022/retrieval_hdf/intpubchem_with_morgan4096_retrieval_db.hdf5"

with open(pred_file, "rb") as f:
    preds = pickle.load(f)

pred_inds = [inds[0] for inds in preds["ranking"]]
names = preds["names"].tolist()
print(names)
print(pred_inds)

db = h5py.File(hdf_file, "r")
pred_fp = [db["fingerprints"][i] for i in pred_inds]
pred_fp_bs = ["".join(fp.astype(str)) for fp in pred_fp]
pred_fp = [
    DataStructs.cDataStructs.CreateFromBitString(fp) for fp in pred_fp_bs
]

print(casmi_mols["name"].tolist())

# Add to casmi_mols dataframe based on names
casmi_mols["pred_fp"] = None
for i, name in enumerate(names):
    casmi_mols.at[
        casmi_mols.index[casmi_mols["name"] == name][0], "pred_fp"
    ] = pred_fp[i]

# Find the most similar molecule in the training set for each CASMI molecule by Tanimoto similarity
casmi_mols["pred_train_most_similar"] = None
casmi_mols["pred_train_similarity"] = None
for i, row in casmi_mols.iterrows():
    fp = row["pred_fp"]
    similarities = train_mols["fp"].apply(
        lambda x: DataStructs.TanimotoSimilarity(fp, x)
    )
    max_sim = similarities.max()
    max_idx = similarities.idxmax()
    casmi_mols.at[i, "pred_train_most_similar"] = train_mols.loc[
        max_idx, "smiles"
    ]
    casmi_mols.at[i, "pred_train_similarity"] = max_sim

In [ ]:
# Plot the distribution of similarities
plt.hist(casmi_mols["pred_train_similarity"], bins=20)
plt.xlabel("Tanimoto similarity")
plt.ylabel("Count")
plt.title("Similarity between predicted FP and training set")
plt.show()

## pred FP vs true FP

In [ ]:
# Find the most similar molecule in the CASMI set for each CASMI molecule by Tanimoto similarity
casmi_mols["pred_true_most_similar"] = None
casmi_mols["pred_true_similarity"] = None
for i, row in casmi_mols.iterrows():
    fp = row["pred_fp"]
    similarities = casmi_mols["fp"].apply(
        lambda x: DataStructs.TanimotoSimilarity(fp, x)
    )
    max_sim = similarities.max()
    max_idx = similarities.idxmax()
    casmi_mols.at[i, "pred_true_most_similar"] = casmi_mols.loc[
        max_idx, "smiles"
    ]
    casmi_mols.at[i, "pred_true_similarity"] = max_sim

In [ ]:
# Plot the distribution of similarities
plt.hist(casmi_mols["pred_true_similarity"], bins=20)
plt.xlabel("Tanimoto similarity")
plt.ylabel("Count")
plt.title("Similarity between predicted FP and true FP")
plt.show()

In [ ]:
casmi_mols.to_pickle("../../results/casmi_fp_analysis/casmi_mols_df.pkl")
casmi_mols.to_csv("../../results/casmi_fp_analysis/casmi_mols.tsv", sep="\t")

## True-train vs pred-train

In [ ]:
plt.style.use(["seaborn-white", "seaborn-paper"])
plt.rc("font", family="sans serif")
# sns.set_palette(['#9e0059', '#6da7de', '#ee266d', '#dee000', '#eb861e'])
sns.set_context("paper", font_scale=1.3)

In [ ]:
# count number of molecules where pred similarity is higher than pred-train similarity
print(
    f"below diagonal: {sum(casmi_mols['train_true_similarity'] > casmi_mols['pred_train_similarity'])}"
)
print(
    f"above diagoneal: {sum(casmi_mols['train_true_similarity'] < casmi_mols['pred_train_similarity'])}"
)
print(
    f"on diagonal: {sum(casmi_mols['train_true_similarity'] == casmi_mols['pred_train_similarity'])}"
)

# Scatterplot tanimoto similarities true-train vs pred-train
sns.jointplot(
    data=casmi_mols,
    x="train_true_similarity",
    y="pred_train_similarity",
    kind="scatter",
    marginal_kws=dict(bins=np.arange(0, 1.01, 0.1)),
)
plt.plot([0, 1.01], [0, 1.01], "k--")
plt.xlim([0, 1.01])
plt.ylim([0, 1.01])
plt.xlabel("Tanimoto similarity of true FP and NN in training set")
plt.ylabel("Tanimoto similarity of predicted FP and NN in training set")
plt.show()

## Pred-true vs pred-train

In [ ]:
# count number of molecules where pred similarity is higher than pred-train similarity
print(
    f"below diagonal: {sum(casmi_mols['pred_true_similarity'] > casmi_mols['pred_train_similarity'])}"
)
print(
    f"above diagoneal: {sum(casmi_mols['pred_true_similarity'] < casmi_mols['pred_train_similarity'])}"
)
print(
    f"on diagonal: {sum(casmi_mols['pred_true_similarity'] == casmi_mols['pred_train_similarity'])}"
)
# Scatterplot tanimoto similarities true-train vs pred-train
sns.jointplot(
    data=casmi_mols,
    x="pred_true_similarity",
    y="pred_train_similarity",
    kind="scatter",
    # hue="Superclass",
    marginal_kws=dict(bins=np.arange(0, 1.01, 0.1)),
)
plt.plot([0, 1.01], [0, 1.01], "k--")
plt.xlim([0, 1.01])
plt.ylim([0, 1.01])
plt.xlabel("Tanimoto similarity predicted vs. true")
plt.ylabel("Tanimoto similarity predicted vs. nearest training")
plt.legend(bbox_to_anchor=(1.2, 1.05))
plt.show()

In [ ]:
# plot violin plot of similarities per superclass

superclass_sim = casmi_mols.melt(
    id_vars="Superclass",
    value_vars=["pred_train_similarity", "pred_true_similarity"],
    var_name="similarity_type",
    value_name="similarity",
)

# drop superclass with only 1 molecule
superclass_sim = superclass_sim.groupby("Superclass").filter(
    # len(x) > 2 because we have 2 similarity types
    lambda x: len(x)
    > 2
)

superclass_sim["similarity"] = superclass_sim["similarity"].astype(float)
fig = plt.figure(figsize=(12, 6))
ax = plt.gca()
sns.violinplot(
    data=superclass_sim,
    x="Superclass",
    y="similarity",
    hue="similarity_type",
    palette="tab10",
    split=False,
    cut=0,
    ax=ax,
)
# add number of molecules in each superclass to tick labels between brackets
superclass_counts = casmi_mols.groupby("Superclass").size()
superclass_counts = superclass_counts[superclass_sim["Superclass"].unique()]
superclass_counts = superclass_counts.reset_index()
superclass_counts["Superclass"] = (
    superclass_counts["Superclass"]
    + " ("
    + superclass_counts[0].astype(str)
    + ")"
)
plt.xticks(
    ticks=range(len(superclass_counts)), labels=superclass_counts["Superclass"]
)
plt.xticks(rotation=90)
plt.ylabel("Tanimoto similarity")
plt.xlabel("Superclass")
plt.ylim([0, 1])
h, l = ax.get_legend_handles_labels()
ax.legend(
    h[:2],
    ["Annotated vs. nearest training", "Annotated vs. true"],
    loc="lower center",
    ncol=2,
)
# plt.show()

save_name = "../../results/casmi_fp_analysis/casmi_fp_analysis_violin.pdf"
plt.savefig(
    save_name, format="pdf", bbox_inches="tight", dpi=600, transparent=True
)

In [ ]:
superclass_sim